In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import DistanceMetric

In [2]:
df = pd.read_csv('heart.csv')

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


#### For people who may not have access to their own medical data 

In [4]:
#age, sex, thalach using formulas to cal
# To estimate your maximum age-related heart rate, subtract your age from 220. For example, for a 50-year-old person, the estimated maximum age-related heart rate would be calculated as 220 – 50 years = 170 beats per minute (bpm).

In [5]:
#54, m

In [6]:
df_2 = df.groupby(by = ['age', 'sex']).mean().reset_index()
df_2[(df_2['sex']==1)& (df_2['age'] ==54)].head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
37,54,1,0.710526,127.184211,253.552632,0.0,0.368421,139.315789,0.394737,1.218421,1.263158,0.736842,2.631579,0.447368


In [7]:
df.chol.describe()
# 1 = male, 0 = female
df1 = df.groupby(by = ['age', 'sex'])['cp', 'trestbps', 'chol', 'fbs', 'restecg', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'].mean()
df_2[(df_2['sex'] == 1) & (df_2['age'] == 54)].reset_index()

,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,37,54,1,0.710526,127.184211,253.552632,0.0,0.368421,139.315789,0.394737,1.218421,1.263158,0.736842,2.631579,0.447368


In [8]:
df1

cp    trestbps        chol       fbs   restecg     exang  \
age sex                                                                   
29  1    1.000000  130.000000  204.000000  0.000000  0.000000  0.000000   
34  0    1.000000  118.000000  210.000000  0.000000  1.000000  0.000000   
    1    3.000000  118.000000  182.000000  0.000000  0.000000  0.000000   
35  0    0.000000  138.000000  183.000000  0.000000  1.000000  0.000000   
    1    0.363636  122.363636  218.727273  0.000000  0.727273  0.636364   
...           ...         ...         ...       ...       ...       ...   
70  1    0.642857  146.285714  251.857143  0.000000  0.500000  0.500000   
71  0    1.000000  124.363636  232.909091  0.363636  0.636364  0.000000   
74  0    1.000000  120.000000  269.000000  0.000000  0.000000  1.000000   
76  0    2.000000  140.000000  197.000000  0.000000  2.000000  0.000000   
77  1    0.000000  125.000000  304.000000  0.000000  0.000000  1.000000   

          oldpeak     slope        ca      thal    target  
age sex                                                    
29  1    0.000000  2.000000  0.000000  2.000000  1.000000  
34  0    0.700000  2.000000  0.000000  2.000000  1.000000  
    1    0.000000  2.000000  0.000000  2.000000  1.000000  
35  0    1.400000  2.000000  0.000000  2.000000  1.000000  
    1    0.581818  1.636364  0.000000  2.636364  0.363636  
...           ...       ...       ...       ...       ...  
70  1    2.050000  0.928571  1.071429  2.500000  0.214286  
71  0    0.690909  1.636364  0.909091  2.000000  1.000000  
74  0    0.200000  2.000000  1.000000  2.000000  1.000000  
76  0    1.100000  1.000000  0.000000  2.000000  1.000000  
77  1    0.000000  2.000000  3.000000  2.000000  0.000000  

[73 rows x 11 columns]

### Removing Outliers

In [9]:
q1 = df.chol.quantile(0.25)
q3 = df.chol.quantile(0.75)
IQR = q3 - q1
lower_limit = q1 - 1.5 * IQR
upper_limit = q3 + 1.5 * IQR
df=df[df["chol"] < upper_limit]

In [10]:
q1 = df.trestbps.quantile(0.25)
q3 = df.trestbps.quantile(0.75)
IQR = q3 - q1
lower_limit = q1 - 1.5 * IQR
upper_limit = q3 + 1.5 * IQR
df=df[df["trestbps"] < upper_limit]

In [11]:
q1 = df.thalach.quantile(0.25)
q3 = df.thalach.quantile(0.75)
IQR = q3 - q1
lower_limit = q1 - 1.5 * IQR
df=df[df["thalach"] > lower_limit]

In [12]:
oldpeak_q1 = df.oldpeak.quantile(0.25)
oldpeak_q3 = df.oldpeak.quantile(0.75)
oldpeak_IQR = oldpeak_q3 - oldpeak_q1
oldpeak_lower_limit = oldpeak_q1 - 1.5 * oldpeak_IQR
oldpeak_upper_limit = oldpeak_q3 + 1.5 * oldpeak_IQR
df=df[df["oldpeak"] < oldpeak_upper_limit]

### Splitting Data and Scaling

In [13]:
y = df["target"]
x = df.drop("target", axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42)

In [14]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

### XGB Machine Learning Code

In [15]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [16]:
knn_parameters = {'weights':('uniform', 'distance'), 'n_neighbors':[1, 2, 3, 4, 5, 6, 7, 8, 9 ,10, 11, 12, 13, 14, 15], 'algorithm':('auto', 'ball_tree', 'kd_tree', 'brute'), 'p':[1, 2, 3, 4, 5], 'metric': ('minkowski', 'euclidean', 'manhattan', 'chebyshev', 'wminkowski', 'seuclidean', 'mahalanobis') }

In [17]:
knn_gs = GridSearchCV(KNN(), knn_parameters, scoring = 'recall')

In [18]:
knn_gs.fit(x_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
                         'metric': ('minkowski', 'euclidean', 'manhattan',
                                    'chebyshev', 'wminkowski', 'seuclidean',
                                    'mahalanobis'),
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15],
                         'p': [1, 2, 3, 4, 5],
                         'weights': ('uniform', 'distance')},
             scoring='recall')

In [19]:
KNN_Model = KNN(n_neighbors=7, weights='distance')

In [20]:
KNN_Model.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=7, weights='distance')

In [21]:
KNN_yhat = KNN_Model.predict(x_test)

In [22]:
print('Recall score for the KNN model is ' + 
str(recall_score(y_test,KNN_yhat)))

Recall score for the KNN model is 0.9444444444444444


In [23]:
from xgboost import XGBClassifier

In [24]:
xgb_model = XGBClassifier(booster='gbtree', max_depth=6, sampling_method='uniform')
xgb_model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [25]:
xgb_y_hat = xgb_model.predict(x_test)

In [26]:
#function that takes in age and sex and returns a row of the mean data from that groupby df

In [27]:
df_mean_data = df.groupby(by = ['age', 'sex']).mean().reset_index()
df_mean_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,29,1,1.000000,130.000000,204.000000,0.000000,0.000000,202.000000,0.000000,0.000000,2.000000,0.000000,2.000000,1.000000
1,34,0,1.000000,118.000000,210.000000,0.000000,1.000000,192.000000,0.000000,0.700000,2.000000,0.000000,2.000000,1.000000
2,34,1,3.000000,118.000000,182.000000,0.000000,0.000000,174.000000,0.000000,0.000000,2.000000,0.000000,2.000000,1.000000
3,35,0,0.000000,138.000000,183.000000,0.000000,1.000000,182.000000,0.000000,1.400000,2.000000,0.000000,2.000000,1.000000
4,35,1,0.363636,122.363636,218.727273,0.000000,0.727273,153.090909,0.636364,0.581818,1.636364,0.000000,2.636364,0.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,70,1,0.642857,146.285714,251.857143,0.000000,0.500000,121.500000,0.500000,2.050000,0.928571,1.071429,2.500000,0.214286
68,71,0,1.000000,124.363636,232.909091,0.363636,0.636364,136.909091,0.000000,0.690909,1.636364,0.909091,2.000000,1.000000
69,74,0,1.000000,120.000000,269.000000,0.000000,0.000000,121.000000,1.000000,0.200000,2.000000,1.000000,2.000000,1.000000
70,76,0,2.000000,140.000000,197.000000,0.000000,2.000000,116.000000,0.000000,1.100000,1.000000,0.000000,2.000000,1.000000


In [28]:
df1=df_mean_data[(df_mean_data['sex'] == 1) & (df_mean_data['age'] == 54)]
df1

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
37,54,1,0.685714,121.628571,251.028571,0.0,0.4,134.542857,0.428571,1.322857,1.2,0.714286,2.6,0.485714


### Returns mean data based on sex and age

In [29]:
def specify():
    
    print("Please Enter Sex")
    ans_sex = input("Sex")
    if ans_sex.lower() in ['male']:
        s = 1
        
    elif ans_sex.lower() in ['female']:
        s = 0
    else:
        exit()
    
    print("Please Enter Age")
    ans_age = int(input("Age"))
    
    if ans_age < 29 or ans_age > 77:
        print("No data for that age")
        exit() 
        
 
    new_df = df_mean_data[(df_mean_data['sex'] == s) & (df_mean_data['age'] == ans_age)]
    return new_df

In [30]:
df_1 = specify()

Please Enter Sex


Sex male

Age 

Please Enter Age


In [31]:
df.age.describe()

count    942.000000
mean      53.933121
std        9.191595
min       29.000000
25%       46.250000
50%       55.000000
75%       60.000000
max       77.000000
Name: age, dtype: float64

minimum age = 29 <br>
maximum age = 77 

### Returns whether or not patient has heart disease based on mean data

In [32]:
def heart_disease(df_1):
    df_1.drop(axis = 1)
    outcome = xgb_model.predict(df_1)
    return outcome

In [33]:
heart_disease(df_1)

ValueError: Need to specify at least one of 'labels', 'index' or 'columns'